# Knowledge graph example notebook
#### This notebook is intended for easier development of our chatbot and is mostly intended as a stop-gap solution to validate the rag components before implementing them into the front end

In [2]:
from setting import RAGSettings
from langchain_community.chains.graph_qa.base import GraphQAChain
from langchain_community.chains.graph_qa.cypher import GraphCypherQAChain
from langchain_community.vectorstores import Neo4jVector
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import (
    create_history_aware_retriever,
    create_retrieval_chain,
)

from langchain_ollama.llms import OllamaLLM
from langchain_neo4j import Neo4jGraph
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph

"""required packages:
- langchain
- langchain-ollama
"""

'required packages:\n- langchain\n- langchain-ollama\n'

In [3]:
check_components_are_running = True


SYSTEM_PROMPT_RAG_EN = """From now on you are an expert on movies. 
The way you garner your knowledge is by generating Cyper query language statements to query a database.
This should be your only ouput"""

setting = RAGSettings()
host = "localhost"

In [15]:
# initialize ollama

cypher_llm = OllamaLLM(model="llama3.1:8b",
                system_prompt=SYSTEM_PROMPT_RAG_EN,
                base_url=f"http://{host}:{setting.ollama.port}",
                temperature=setting.ollama.temperature,
                context_window=setting.ollama.context_window,
                request_timeout=setting.ollama.request_timeout,
)

qa_llm = OllamaLLM(model="llama3.1:8b",
                # system_prompt=SYSTEM_PROMPT_RAG_EN,
                base_url=f"http://{host}:{setting.ollama.port}",
                temperature=setting.ollama.temperature,
                context_window=setting.ollama.context_window,
                request_timeout=setting.ollama.request_timeout,
)


In [16]:
# Initialize graph and LLM
graph = Neo4jGraph(url=f"bolt://{host}:7687", username="neo4j", password="password", database="neo4j")

In [17]:
if check_components_are_running:
    print("Ollama response: ", cypher_llm.invoke("hello"))
    print("Graph response: ", graph.query("MATCH (n) RETURN n LIMIT 1"))

Ollama response:  Hello! How are you today? Is there something I can help you with or would you like to chat?
Graph response:  [{'n': {'tagline': 'Welcome to the Real World', 'title': 'The Matrix', 'released': 1999}}]


In [18]:
# Create the chain itself
chain = GraphCypherQAChain.from_llm(
    qa_llm=qa_llm,
    cypher_llm=cypher_llm,
    graph=graph,
    allow_dangerous_requests=True,
    verbose=True
)

In [19]:
# Query the system
response = chain.run("What movies were released in the year 1999?")
print("Response from the LLM: ")
print(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie) WHERE m.released = 1999 RETURN m
Full Context:
[{'m': {'tagline': 'Welcome to the Real World', 'title': 'The Matrix', 'released': 1999}}, {'m': {'tagline': 'First loves last. Forever.', 'title': 'Snow Falling on Cedars', 'released': 1999}}, {'m': {'tagline': "Walk a mile you'll never forget.", 'title': 'The Green Mile', 'released': 1999}}, {'m': {'tagline': "One robot's 200 year journey to become an ordinary man.", 'title': 'Bicentennial Man', 'released': 1999}}]

> Finished chain.
Response from the LLM: 
The Matrix, Snow Falling on Cedars, The Green Mile, Bicentennial Man was released in the year 1999.
